<a href="https://colab.research.google.com/github/lucifernandes/dadosSociaisSVM/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pré processamento

##Dados

In [ ]:
import pandas as pd
base = pd.read_csv('/content/drive/My Drive/Bases/base_v01.csv', sep=',', decimal=',')
base = base.dropna(how="any") 
base = base.reset_index(drop=True)
base.head()

In [ ]:
base['renda'] = (base['renda']).astype(int)

dadosSocio = base.loc[:, ['idade', 'sM', 'sF', 'corBr',	'corPr','corPa','corAm','corIn']]
cbc = base.loc[:, ['carro','empreg','maqlav','dvd','banhe','gelad','freezer','comput',
                          'lavlou','microon','motocic','secadora','agua','rua','escola','renda']]
bloco1 = base.loc[:,['p41aa','p41ab','p41ad','p41ae','p41af','p41ag','p41ah','p41ai',
                     'p41aj','p41ak','p41al','p41am','p41an','p41ao','p41ap']]
bloco2 = base.loc[:,['p42a','p42b','p42d','p42e','p42f','p42g','p42h','p42i','p42j','p42k',
                     'p42l','p42m','p42n','p42o','p42p']]
bloco3 = base.loc[:,['p43a','p43b','p43d','p43f','p43g','p43h','p43i','p43j','p43k','p43l',
                     'p43m','p43n','p43o','p43p']]
#------------------------------- índice Autoritarismo ------------------------------------------------
#Sentenças do 4º bloco do questionário
bloco4 = base.loc[:, ['INV1p45a','INV1p45b','INV1p45c','INV1p45d','INV1p45e','INV1p45f',
                      'INV1p45g','INV1p45h','INV1p45i','INV1p45j','INV1p45k','INV1p45l',
                      'INV1p45m','INV1p45n','INV1p45o','INV1p45p','INV1p45q']]  
#Adicionando à base a coluna 'IndAuto' com os valores dos índices de cada amostra                   
#A = bloco4.mean(axis=1)
#B = bloco1.mean(axis=1)
#Cálculo do índice médio  
mediana = bloco4.mean(axis=1).median()
#mediana = bloco1.mean(axis=1).median()
#Para valores abaixo ou igual a média, recebem valor 0. Para valores acima da média, recebem valor 1
#A = A.values
print(mediana)

auto = pd.DataFrame({})
auto['IndAuto'] = bloco4.mean(axis=1)
#auto['Indbloco1'] = bloco1.mean(axis=1)

for i in range(len(auto)):
  if auto.iloc[i,0] < mediana:
    auto.iloc[i,0] = 0
  else:
    auto.iloc[i,0] = 1

auto['IndAuto'] = (auto['IndAuto']).astype(int)
#auto['Indbloco1'] = (auto['Indbloco1']).astype(int)
#base['IndAuto'] = A
#base['IndAuto'] = (base['IndAuto']).astype(int)

4.411764705882353


In [ ]:
prev = pd.DataFrame({})
prev['cbc'] = cbc.sum(axis=1)
prev['bloco1'] = bloco1.mean(axis=1)
#prev['auto'] = bloco4.mean(axis=1)
prev['bloco2'] = bloco2.mean(axis=1)
prev['bloco3'] = bloco3.mean(axis=1)
prev['idade'] = dadosSocio.loc[:,'idade']
prev['sM'] = dadosSocio.loc[:,'sM']
prev['sF'] = dadosSocio.loc[:,'sF']
prev['corBr'] = dadosSocio.loc[:,'corBr']
prev['corPr'] = dadosSocio.loc[:,'corPr']
prev['corPa'] = dadosSocio.loc[:,'corPa']
prev['corAm'] = dadosSocio.loc[:,'corAm']
prev['corIn'] = dadosSocio.loc[:,'corIn']
#prev['classe'] = auto

classe = pd.DataFrame({})
classe['classe'] = auto['IndAuto']

#classe['classe'].value_counts()

prev.head()

##base2

In [ ]:
c0 = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])
c1 = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])

for i in range(len(prev)):
  if prev.loc[i,'classe'] == 0:
    c0.loc[i] = prev.iloc[i,:]
  else:
    c1.loc[i] = prev.iloc[i,:]
  
c0 = c0.reset_index(drop=True)
c1 = c1.reset_index(drop=True)  

from random import sample
import numpy as np
from array import array 
r0 = np.arange(1581)       
r = sample(list(r0),176)



c1n = pd.DataFrame({}, columns=['cbc','bloco1','bloco2','bloco3','idade','sM','sF','corBr','corPr','corPa','corAm','corIn','classe'])
for i in r:
  c1n.loc[i] = c1.iloc[i,:]
c1n = c1n.reset_index(drop=True)  

data = pd.concat([c0, c1n])     #Concatenação dos sinais da Classe 1 com os sinais da Classe 2
data.reset_index(drop=True,inplace=True) #Resetando os índices


clas = data.loc[:,'classe']
data = data.drop(columns=['classe'])

clas.head(352)

##classificador

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

resultados20 = []
for i in range(20):
  
  previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(data, clas, test_size=0.10, random_state=i)
      #print(previsores_teste)

  classificador = SVC(kernel= 'rbf', random_state = i, C= 2.0)

  classificador.fit(previsores_treinamento, classe_treinamento)
  previsoes = classificador.predict(previsores_teste)

  
  precisao = accuracy_score(classe_teste, previsoes)
  matriz = confusion_matrix(classe_teste, previsoes)
    #print(precisao)
    #print(previsoes)

  resultados20.append(precisao)
  #print(str(precisao).replace('.', ','))

for i in range(20):
  print(str(resultados20[i]).replace('.', ','))







0,6388888888888888
0,6944444444444444
0,7222222222222222
0,5555555555555556
0,6111111111111112
0,7777777777777778
0,6111111111111112
0,6666666666666666
0,5277777777777778
0,5
0,6111111111111112
0,5833333333333334
0,5555555555555556
0,6666666666666666
0,5277777777777778
0,6388888888888888
0,6666666666666666
0,5833333333333334
0,6944444444444444
0,6388888888888888
